In [1]:
!pip install git+https://github.com/PrithivirajDamodaran/Parrot_Paraphraser.git

  Cloning https://github.com/PrithivirajDamodaran/Parrot_Paraphraser.git to /tmp/pip-req-build-aira2xr8
  Running command git clone --filter=blob:none --quiet https://github.com/PrithivirajDamodaran/Parrot_Paraphraser.git /tmp/pip-req-build-aira2xr8
  Resolved https://github.com/PrithivirajDamodaran/Parrot_Paraphraser.git to commit 720a87a1ee557d8ed8d9a021adbdd1dd5616c5f9
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 kB 3.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 21.4 MB/s eta 0:00:00a 0:00:01
  Created wheel for parrot: filename=parrot-1.0-py3-none-any.whl size=8591 sha256=c7b4748b627d2cb69ee86e239601ee678a6852689508ef049cdaf3c01b4d6481
  Stored in directory: /tmp/pip-ephem-wheel-cache-1u30vxhu/wheels/b1/4b/8a/025204648edd605a2ad5be4c3520af93b46a06ee8c4e9d6262
Successfully built parrot


In [2]:
from parrot import Parrot
import torch
import warnings
import pandas as pd
import tqdm
warnings.filterwarnings("ignore")

In [3]:
parrot = Parrot(model_tag="prithivida/parrot_paraphraser_on_T5", use_gpu=True)

Downloading:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.34k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.74k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [39]:
df = pd.read_csv("train_data.csv")
# get all rows whose label is 1
phrases = df[df["label"] == 1]["text"].tolist()
print("Number of phrases: ", len(phrases))

Number of phrases:  706


In [5]:
new_df = pd.DataFrame(columns=["text", "label"])
# use tqdm to show progress bar
for phrase in tqdm.tqdm(phrases):
    if phrase:
        para_phrases = parrot.augment(input_phrase=phrase)
        if para_phrases:
            for para_phrase in para_phrases:
                # write para_phrase to df as a new row whose label is 1
                new_df = new_df.append({"text": para_phrase, "label": 1}, ignore_index=True)


100%|██████████| 706/706 [2:49:41<00:00, 14.42s/it]  


In [6]:
# save the new_df to a csv file
new_df.to_csv("augmented_data.csv", index=False)

In [19]:
# get the first row of the new_df
new_df.iloc[1]['text'][0]

'" That is something that has been a tremendous help to us and these young men and women are very appreciative of what have been done for them , " he said .'

In [20]:
len(new_df)

1393

In [28]:
df1 = pd.DataFrame(columns=["text", "label"])
for i in range(len(new_df)):
    df1 = df1.append({"text": new_df.iloc[i]['text'][0], "label": 1}, ignore_index=True)

In [40]:
original_ones = df[df["label"] == 1]
ones = pd.concat([original_ones, df1], ignore_index=True)

In [41]:
len(ones)

2099

In [31]:
df1.to_csv("augmented_data.csv", index=False)

In [42]:
zeros = df[df["label"] == 0]
# downsample the zeros to 3000 rows
zeros = zeros.sample(n=4198)

In [43]:
# concatenate the original df and the df1
res = pd.concat([zeros, ones], ignore_index=True)
res.to_csv("train_data_balanced.csv", index=False)

In [44]:
len(res)

6297